 # TP 6: Apprentissage profond(Deep learning)
 ## TP 6.4  : Optimisation solution naive

- Régularisation :  Dropout
- Normalisation
    - Inputs
    - les autres couches (batch Normalisation: avant ou après l'activation)
- Optimiseurs:
    - RMSProp, Adam, SGD,
- Hyperparameter tuning :
    - learning_rate
    - \#couches
    - \#neurones par couche
    - taille du mini batch
- ...
# Exercice
Trouver le meilleur modèle pour les données  de "smoking"
Le programme doit tester les différentes configurations (et hyperparamètres).

Visiter le site keras ou tensorflow pour vérifier la maniène d'utiliser ces différents paramètres

PS : comparer aussi avec les modèles classiques : (KNN, Randomforest, ...)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import seaborn as sns


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Préparation des données

In [4]:
# Lecture du dataset n permet de limiter le nombre de lignes à lire
# Pour faciliter les tests

def load_data(n):
    data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')
    return data[0:n]

def select_variables(data):
    data.dropna(axis=0, inplace=True)
    y = data['smoking'] # récupérer la colonne survived et la mettre dans y
    # récuperer le reste des données dans X utiliser la fonction titanic.drop ???, ??? )
    X = data.drop('smoking', axis=1)
    return X,y

#### Split des données

In [5]:
def split_data(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

#### Normalisation

In [6]:
## On peut utiliser une simple normalisation (x-mu)/ecart type)
def normaliser(X_train, X_test):
    mean = X_train.mean()
    std  = X_train.std()
    X_train = (X_train - mean) / std
    X_test  = (X_test  - mean) / std

    return X_train, X_test

### Définition du modèle
avec prise d'un nombre de couches et nombre de neurones variables

In [7]:
def build_model(nb_layers, nb_units):
    m = X_train.shape[1]
    model = tf.keras.Sequential ()
    model.add(tf.keras.layers.Input(m, name="InputLayer"))
    for l in range(nb_layers):
        model.add(tf.keras.layers.Dense(nb_units,name='layer'+str(l)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation('relu'))
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid',name='output'))
    return model

In [8]:
#Compiler le modèle
#Optimiserer: SGD, AdamW, adadelta, ...
def compiler(model,optimizer,loss,metrics):
    model.compile(optimizer=optimizer,
                  loss= loss,
                  metrics=[metrics])
    return model



### 4. Entrainement du modèle (Model training)

In [9]:
# Entrainement du modele
def train(model, X_train, y_train,epochs,batch_size):
    history  = model.fit(X_train,
                     y_train,
                     epochs=epochs,
                     batch_size= batch_size,
                     verbose=False,
                     validation_data = (X_test, y_test))
    return model, history


### 5. Evaluation des performances du modèle


In [10]:
def evaluer(model, X_test, y_test):
    score = model.evaluate(X_test, y_test, verbose=0)
    return score

#print('Test loss     :', score[0])
#print('Test accuracy :', score[1])


In [11]:
def visualiser_confusion(model, X_test, y_test):
    y_pred = model.predict(X_test)
    # Get class labels
    y_classes = np.argmax(y_pred, axis=-1)

    cm = confusion_matrix(y_test, y_classes)
    #disp= ConfusionMatrixDisplay(confusion_matrix=cm)

    sns.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
    plt.show()


### Main program

In [13]:
# Préparation ds données
# le -1 du load_data(-1) veut dire on prend toutes les lignes
data=load_data(-1)
# sélectionner les variables
X,y = select_variables(data)
X_train, X_test, y_train, y_test = split_data(X,y)
X_train, X_test = normaliser(X_train, X_test)
X_train.head()

id       age  height(cm)  weight(kg)  waist(cm)  eyesight(left)  \
85326   0.127385 -1.208808    0.536165    0.627257   1.030512       -0.014639   
60700  -0.407954 -0.363404   -1.729719   -1.361019  -0.332274       -1.249787   
44065  -0.769579  1.750108   -1.729719   -1.361019  -1.561016       -0.755728   
39538  -0.867990  0.904703   -1.163248   -0.963364  -0.332274        0.479420   
105427  0.564356  0.904703   -0.596777   -0.565708   0.002837       -0.755728   

        eyesight(right)  hearing(left)  hearing(right)  systolic  ...  \
85326         -0.511698      -0.156478       -0.154652 -0.351960  ...   
60700         -1.275566      -0.156478       -0.154652 -0.194728  ...   
44065         -0.002452      -0.156478       -0.154652  1.298967  ...   
39538          0.506793      -0.156478       -0.154652 -1.059500  ...   
105427        -0.511698      -0.156478       -0.154652  0.591427  ...   

        triglyceride       HDL       LDL  hemoglobin  Urine protein  \
85326       0.220438 -0.635159  0.536704    1.331728       2.659841   
60700      -1.201148  0.295065 -0.195901   -2.086931      -0.213242   
44065      -0.444985  0.795955  0.257616    1.052654      -0.213242   
39538      -1.246518 -0.563603 -0.649419   -0.970634      -0.213242   
105427     -0.671834 -0.134269  1.548396   -0.133411      -0.213242   

        serum creatinine       AST       ALT       Gtp  dental caries  
85326          -0.515541  0.152527  0.478382 -0.518961      -0.497645  
60700          -1.625699 -1.193554 -0.877251 -0.872516      -0.497645  
44065          -1.625699 -0.882920 -0.142950 -0.358254      -0.497645  
39538          -1.625699 -0.675831 -0.651312 -0.647526      -0.497645  
105427         -1.070620 -1.193554 -0.764281 -0.326113      -0.497645  

[5 rows x 23 columns]

In [ ]:
history=[]
loss='BinaryCrossentropy'
metrics='accuracy'
nb_layers=[1,2,4,6]
nb_units=[16, 32]
epochs=[10,50, 100]
batch_size=[16, 32]
optimizers = ['Adam', 'RMSprop', 'SGD', 'Adamax']

best_model=0

for l in nb_layers:
    for u in nb_units:
        model = build_model(l,u)
        for optimize in optimizers:
            for ep in epochs:
                for batch in batch_size:
                    model = compiler(model,optimize,loss,metrics)
                    model,history = train(model, X_train, y_train,ep,batch)
                    score = evaluer(model, X_test, y_test)
                    print('nb_layer',l, 'nb_units' ,u)
                    print('optimiseur',optimize,'Test accuracy  epoch   :',ep, 'batch', batch, score[1])
                    if  score[1] > best_model:
                        best_model = score[1]
                        best_param_model = {"#layers":l,"#units": u, "ep":ep, "batch":batch, "optimize":optimize}


nb_layer 1 nb_units 16
optimiseur Adam Test accuracy  epoch   : 10 batch 16 0.7697089314460754
nb_layer 1 nb_units 16
optimiseur Adam Test accuracy  epoch   : 10 batch 32 0.7694891691207886
nb_layer 1 nb_units 16
optimiseur Adam Test accuracy  epoch   : 50 batch 16 0.7704624533653259
nb_layer 1 nb_units 16
optimiseur Adam Test accuracy  epoch   : 50 batch 32 0.7690496444702148


KeyboardInterrupt: ignored

#### Afficher les paramètres du meilleur modèle

In [ ]:
best_param_model

In [ ]:
best_param_model["#layers"]

#### Play with the best model

In [ ]:
## sur les données de Text (X_test, y_test)
model = build_model(best_param_model["#layers"],best_param_model["#units"])
model = compiler(model,best_param_model["optimize"],loss,metrics)
model,history = train(model, X_train, y_train,best_param_model["ep"],best_param_model["batch"])
score = evaluer(model, X_test, y_test)
print('Test loss     :', score[0])
print('Test accuracy :', score[1])


In [ ]:
print('Test loss     :', score[0])
print('Test accuracy :', score[1])